In [1]:
!pip install torch_geometric
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.6 MB/s eta 0:00:00


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.nn import Linear, Parameter
from torch_geometric.datasets import WebKB, Planetoid, WikipediaNetwork, Actor
from torch_geometric.nn import GCNConv, VGAE
#from torch_geometric.utils import train_test_split_edges
from torch_geometric.transforms import RandomLinkSplit
from torch_geometric.utils import negative_sampling
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree
from torch_geometric.data import Data
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.model_selection import StratifiedKFold
from torch_geometric.loader import DataLoader
from torch import tensor


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
device

device(type='cuda')

In [4]:
#b84ddf302cba5ed974393a1a631bde44eff035a8
import wandb
run_vgae_rewire = wandb.init(
        project = "VAE-Rewire-CL-v6-cv-SHD",
        config = {
            "architecture": "VGAE+Custom_loss-Countineous-crossval",
            "dataset":"texas",
            "model":"gcn",
            "epoch": 500,
            "lr_gcn": 0.01,
            "weight_decay_gcn":1e-3,
            "lr_VGAE": 0.1,
            "weight_decay_VGAE": 5e-3,
            "Batch size": 1,
            "mask":0,
            "alpha":1,
            "beta":1

        }
    )

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: sidgraph (graph-rewire-project). Use `wandb login --relogin` to force relogin


In [5]:
dataset = WebKB(root="/home/siddy/META/data", name="texas")
data = dataset[0].to(device)

In [6]:
edge_index= data.edge_index
num_nodes = data.x.size(0)
adj_matrix = torch.zeros((num_nodes, num_nodes), dtype=torch.long)
for i in range(edge_index.size(1)):
    u, v = int(edge_index[0, i]), int(edge_index[1, i])
    adj_matrix[u, v] = 1
    adj_matrix[v, u] = 1

In [7]:
from collections import defaultdict, deque
edge_index  = data.edge_index
adj_list = defaultdict(list)
for i in range(edge_index.size(1)):
        u, v = int(edge_index[0, i]), int(edge_index[1, i])
        adj_list[u].append(v)
        adj_list[v].append(u)

In [8]:

# below code provides functions that mimics the above code using the einsum operations in the pytorch bbut provides a scaler value instead of a vector
def pos_loss(z, adj_list, edge_index):
    pos_indices = []
    num_nodes = len(adj_list.keys())

    for i in range(edge_index.size(1)):
        u, v = int(edge_index[0, i]), int(edge_index[1, i])
        d_u = len(adj_list[u])
        d_v = len(adj_list[v])
        u_neighbors_norm = torch.stack([z[node] / torch.norm(z[node]) for node in adj_list[u]])
        v_neighbors_norm = torch.stack([z[node] / torch.norm(z[node]) for node in adj_list[v]])

        term_u = torch.einsum('ij,j->', u_neighbors_norm, z[v])
        #term_u /= (torch.norm(z[u]) * torch.norm(z[v]))
        #print(f"term_u shape:{term_u.shape}")
        #print(f"term_u:{term_u}")
        u_ = term_u / d_u

        term_v = torch.einsum('ij, j->', v_neighbors_norm, z[u])
        #term_v /= (torch.norm(z[v]) * torch.norm(z[u]))
        #print(f"term_v{term_v}")
        #print(f"term_v shape:{term_v.shape}")
        v_ = term_v / d_v

        pos_indices.append(u_ + v_)

    return pos_indices

def neg_loss(z, adj_list, edge_index):
    neg_indices = []
    num_nodes = len(adj_list.keys())

    for i in range(edge_index.size(1)):
        u, v = int(edge_index[0, i]), int(edge_index[1, i])
        d_u = len(adj_list[u])
        d_v = len(adj_list[v])

        #u_neighbors = torch.stack([z[node] for node in adj_list[u]])
        #v_neighbors = torch.stack([z[node] for node in adj_list[v]])
        u_neighbors_norm = torch.stack([z[node] / torch.norm(z[node]) for node in adj_list[u]])
        v_neighbors_norm = torch.stack([z[node] / torch.norm(z[node]) for node in adj_list[v]])

        for u in u_neighbors_norm:
          for v in v_neighbors_norm:
            z_u_dot_z_v = torch.matmul(u, v.T)
            #norm = z_u_dot_z_v/(torch.norm(u) * torch.norm(v))
        term_ = torch.sum(z_u_dot_z_v) / (d_u * d_v)
        #print(term_)
        neg_indices.append(term_)

    return neg_indices

def custom_objective_2(pos_loss, neg_loss):
  wandb.log({"pos_indices":pos_loss})
  wandb.log({"neg_indices": neg_loss})
  return (pos_loss+2) / 2*(neg_loss+1)

In [ ]:
# # def neg_loss(z, adj_matrix, edge_index):
# #     neg_indices = []
# #     num_nodes = adj_matrix.size(0)
# #     for i in range(edge_index.size(1)):
# #         u, v = int(edge_index[0, i]), int(edge_index[1, i])
# #         Iv, Iu = torch.zeros(num_nodes), torch.zeros(num_nodes)
# #         Iv[v] = 1
# #         Iu[u] = 1
# #         Iv = Iv.unsqueeze(1)  # Convert Iv to a column vector
# #         Iu = Iu.unsqueeze(1)  # Convert Iu to a column vector
# #         print(f"Iv shape: {Iv.shape}")
# #         print(f"Iv shape: {Iu.shape}")
# #         adj_row_u = adj_matrix[u]
# #         adj_row_v = adj_matrix[v]
# #         print(f"adj_u_shape: {adj_row_u.shape}")
# #         print(f"adj_v_shape: {adj_row_v.shape}")
# #         adj_row_u = adj_row_u.unsqueeze(1)
# #         adj_row_v = adj_row_v.unsqueeze(1)
# #         sum_adj_u = torch.sum(adj_row_u)
# #         sum_adj_v = torch.sum(adj_row_v)
# #         print(f"(Iu * z)-shape:{(Iu * z).shape}")
# #         term_1 = (torch.matmul((adj_row_u * z).t(), (Iv * z)) + torch.matmul((adj_row_v * z).t(), (Iu * z))) / (sum_adj_u + sum_adj_v)
# #         print(f"adj_v_shape:{term_1.shape}")
# #         neg_indices.append(term_1)
# #     return neg_indices

# def neg_loss(z, adj_matrix, edge_index):
#     neg_indices = []
#     num_nodes = adj_matrix.size(0)
#     for i in range(edge_index.size(1)):
#         u, v = int(edge_index[0, i]), int(edge_index[1, i])
#         Iv, Iu = torch.zeros(num_nodes), torch.zeros(num_nodes)
#         Iv[v] = 1
#         Iu[u] = 1
#         Iv = Iv.unsqueeze(1)
#         Iu = Iu.unsqueeze(1)

#         adj_row_u = adj_matrix[u]
#         adj_row_v = adj_matrix[v]

#         adj_row_u = adj_row_u.unsqueeze(1)
#         adj_row_v = adj_row_v.unsqueeze(1)
#         sum_adj_u = torch.sum(adj_row_u)
#         sum_adj_v = torch.sum(adj_row_v)

#         term_1 = torch.einsum('ij,ij->', adj_row_u * z, Iv * z) + torch.einsum('ij,ij->', adj_row_v * z, Iu * z)
#         term_1 /= (sum_adj_u + sum_adj_v)

#         neg_indices.append(term_1.item())

#     return neg_indices


# def pos_loss(z, adj_matrix, edge_index):
#     pos_indices = []
#     for i in range(edge_index.size(1)):
#         u, v = int(edge_index[0, i]), int(edge_index[1, i])
#         adj_row_u = adj_matrix[u]
#         adj_row_v = adj_matrix[v]
#         adj_row_u = adj_row_u.unsqueeze(1)
#         adj_row_v = adj_row_v.unsqueeze(1)
#         sum_adj_u = torch.sum(adj_row_u)
#         sum_adj_v = torch.sum(adj_row_v)
#         # term_2 = (torch.matmul((adj_row_u * z).t(), (adj_row_v * z))) / (sum_adj_u * sum_adj_v)
#         term_2 = torch.einsum('ij,ij->', adj_row_u * z, adj_row_v * z) / (sum_adj_u * sum_adj_v)
#         pos_indices.append(term_2.item())
#     return pos_indices

In [ ]:
class GCNConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super().__init__(aggr='add')
        self.lin = Linear(in_channels, out_channels, bias=False)
        self.bias = Parameter(torch.empty(out_channels))

        self.reset_parameters()

    def reset_parameters(self):
        self.lin.reset_parameters()
        self.bias.data.zero_()

    def forward(self, x, edge_index):
        # what is the shape of inpur x ? - needed [N, in_channels]
        # edge indices shape needed is [2, E]

        #add self_loops to the adjacency matrix, how to give num nodes?
        #edge_index, _ = add_self_loops(edge_index)
        #print(edge_index)
        # linearly transform node feature matrix
        x = self.lin(x)
        #x = torch.index_select(input=x, index=edge_index[0], dim=0)
        # x_ball = torch.cat([torch.index_select(input=x, index=edge_index[0], dim=0), NOTE THAT IT WILL GIVE INDEX OUT OF RANGE ONE OPTION IS TO GO WITH REINDEXING
        #             torch.index_select(input=x, index=edge_index[1], dim=0)],dim=0)
        #compute normalization
        row, col = edge_index
        deg = degree(col, x.size(0), dtype=x.dtype)
        deg_inv_sqrt = deg.pow(0.5)
        deg_inv_sqrt[deg_inv_sqrt==float('inf')] = 0
        #print(deg_inv_sqrt.shape)
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]

        # propagating messages
        out = self.propagate(edge_index, x=x, norm=norm)
        #out = torch.index_select(input=out, index=min(edge_index[0]), dim=0) #NOTE TRICK IS TO PICK MIN EDGE INDEX AS IT WILL CORRESPOND TO THE CENTER NODE OF THE BALL
        # bias
        out += self.bias
        return torch.squeeze(out)

    def message(self, x_j, norm):
        # x_j has shape [E, out_channels]
        # normalize node features
        return norm.view(-1,1) *x_j

In [9]:
class GCNNet(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv = GCNConv(in_channels, hidden_channels)
        self.fc = Linear(hidden_channels, out_channels)
    def forward(self, x, edge_index):
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.fc(self.conv(x, edge_index))
        return x

In [11]:
# alpha, beta = 1.0, 1.0
# def custom_objective(sigma_1, sigma_2):
#   # sigma1 = alpha*((sigma_1-0.5)**2)
#   # sigma2 = beta*(sigma_2)
#   wandb.log({"neg_indices":sigma_1})
#   wandb.log({"pos_indices":sigma_2})
#   return sigma_1+(1-sigma_2)

In [10]:
# Encoder
class GCNEncoder(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCNEncoder, self).__init__()
        self.conv1 = GCNConv(in_channels, 2 * out_channels)
        self.conv_mu = GCNConv(2 * out_channels, out_channels)
        self.conv_logstd = GCNConv(2 * out_channels, out_channels)

    def forward(self,x ,edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv_mu(x, edge_index), self.conv_logstd(x, edge_index)

In [11]:
out_channels= dataset.num_classes
num_features = dataset.num_features
epochs=500
num_nodes_train = data.x.size(0)
# neg_indices = neg_index(data.edge_index, adj_matrix, adj_squared)
# pos_indices = pos_index(data.edge_index, adj_matrix, adj_squared)

In [12]:
model = VGAE(GCNEncoder(num_features, out_channels))
gcn_net = GCNNet(in_channels=data.x.size(1), hidden_channels=64, out_channels=dataset.num_classes)
gcn_net = gcn_net.to(device)
model = model.to(device)
x = data.x.to(device)
optimizer1 = torch.optim.Adam(model.parameters(), lr=0.1, weight_decay=0.001)
optimizer2 = torch.optim.Adam(gcn_net.parameters(), lr=0.01, weight_decay=1e-3)

In [13]:
#Generator function for k-fold dataset
def cross_validation_with_test_set(dataset, folds):
    cross_val_dataset = []
    for fold, (train_idx, test_idx) in enumerate(zip(*kfold(dataset, folds))):
        data = Data(x=dataset.data.x, edge_index=dataset.data.edge_index, y=dataset.data.y)
        train_mask = torch.zeros([dataset.data.x.shape[0]], dtype=torch.bool)
        test_mask = torch.zeros([dataset.data.x.shape[0]], dtype=torch.bool)
        train_mask[train_idx]=True
        test_mask[test_idx] = True
        data.train_mask = train_mask
        data.test_mask = test_mask
        cross_val_dataset.append(data)
    return cross_val_dataset

# implemented train-test k-fold
def kfold(dataset, folds):
    skf = StratifiedKFold(folds, shuffle=True, random_state=1610)
    train_indices, test_indices = [], []
    for idx, (train_idx, test_idx) in enumerate(skf.split(dataset.data.x, dataset.data.y)):
        train_idx, test_idx = (train_idx, test_idx)
        train_indices.append(torch.from_numpy(train_idx).to(torch.long))
        test_indices.append(torch.from_numpy(test_idx).to(torch.long))
    return train_indices, test_indices

In [14]:
def train(train_data, adj_list):
    model.train()
    gcn_net.train()
    optimizer1.zero_grad()

    # neg_edge_index = negative_sampling(
    #     edge_index= data.edge_index,
    #     num_nodes= data.x.size(0),
    #     num_neg_samples= data.edge_index.size(1)
    # )
    # neg_indices = neg_index(num_nodes_train, train_data.edge_index)
    # pos_indices = pos_index(num_nodes_train, train_data.edge_index)
    z = model.encode(x, data.edge_index)
    print(f"latent space shape: {z.shape}")
    #adj = torch.sigmoid(torch.matmul(z, z.t()))
    #print(f"adj matrix shape: {adj.shape}")
    loss = model.recon_loss(z, train_data.edge_index)
    wandb.log({"recon_loss":loss.item()})
    kl_loss = (1 / data.num_nodes) * model.kl_loss()  # new line
    wandb.log({"kl_loss":kl_loss.item()})
    loss += kl_loss
    #adj_binary = (adj > 0.5).float()
    #edge_list = adj.nonzero(as_tuple=False)
    #edge_list = torch.permute(torch.tensor(edge_list, dtype=torch.long), (1,0)).to(device)
    neg_indices = neg_loss(z, adj_list, train_data.edge_index)
    pos_indices = pos_loss(z, adj_list, train_data.edge_index)
    objective_loss = 0.0
    for i in range(len(neg_indices)):
        sigma_1 = torch.tensor(neg_indices[i], requires_grad=True)
        sigma_2 = torch.tensor(pos_indices[i], requires_grad=True)
        objective = custom_objective_2(sigma_1, sigma_2)
        objective_loss += objective
    #objective_loss /= len(edge_index[0])
    wandb.log({"obj_loss":objective_loss.item()})
    loss += objective_loss
    optimizer2.zero_grad()
    out = gcn_net(data.x, data.edge_index)
    out_2 = gcn_net(data.x, data.edge_index)
    nc_loss = F.cross_entropy(out[data.train_mask], data.y[data.train_mask])
    nc_loss_2 = F.cross_entropy(out_2[data.train_mask], data.y[data.train_mask])
    wandb.log({"nc_loss":nc_loss.item()})
    loss += nc_loss
    loss.backward()
    optimizer1.step()
    nc_loss_2.backward()
    optimizer2.step()
    return nc_loss_2, loss


def test(test_data):
    model.eval()
    gcn_net.eval()
    with torch.no_grad():
        # test_neg_edge_index = negative_sampling(
        #     edge_index= test_data.edge_index,
        #     num_nodes= test_data.x.size(0),
        #     num_neg_samples=test_data.edge_index.size(1)
        # )
        # #z = model.encode(x, test_data.edge_index)
        out = gcn_net(data.x, data.edge_index).argmax(dim=-1)
        accs=[]
        for mask in [data.train_mask, data.test_mask]:
            accs.append(int((out[mask] == data.y[mask]).sum())/ int(mask.sum()))
        return accs

    #return model.test(z, test_data.edge_index, test_neg_edge_index)


In [15]:
folds= 5
epochs=300
cross_val_dataset = cross_validation_with_test_set(dataset, folds)
loader = DataLoader(cross_val_dataset, batch_size=1, shuffle=False)

/home/siddy/anaconda3/envs/GDL/lib/python3.8/site-packages/torch_geometric/data/in_memory_dataset.py:284: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)
/home/siddy/anaconda3/envs/GDL/lib/python3.8/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/home/siddy/anaconda3/envs/GDL/lib/python3.8/site-packages/torch_geometric/data/in_memory_dataset.py:284: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMe

In [17]:
import time
epochs=500
best_val_acc = final_test_acc = 0
times = []
train_losses, val_accs, test_accs = [], [], []
for data in loader:
  data = data.to(device)
  for epoch in range(1, epochs+1):
      start = time.time()
      _, loss= train(data, adj_list)
      wandb.log({"total_loss":loss})
      train_losses.append(loss)
      train_acc, test_acc = test(data)
      val_accs.append(train_acc)
      test_accs.append(test_acc)
      print(f"Epoch:{epoch}, Loss:{loss}, Train:{train_acc}, Test:{test_acc}")
      wandb.log({"train_acc":train_acc})
      wandb.log({"test_acc":test_acc})

loss, acc = tensor(train_losses), tensor(test_accs)
loss, acc = loss.view(folds, epochs), acc.view(folds, epochs)
print(loss.shape, acc.shape)

loss, argmin = loss.min(dim=1)
acc = acc[torch.arange(folds, dtype=torch.long), argmin]
print(loss, acc)
loss_mean = loss.mean().item()
acc_mean = acc.mean().item()
acc_std = acc.std().item()

print(f"train_loss:{loss_mean:.4f}, Test Acuracy: {acc_mean:.3f}" f"+- {acc_std:.3f}")
wandb.log({"mean_train_loss": loss_mean})
wandb.log({"mean_test_acc": acc_mean})
wandb.log({"test_std": acc_std})

run_vgae_rewire.finish()

latent space shape: torch.Size([183, 5])


/tmp/ipykernel_11037/1641927140.py:45: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3277.)
  z_u_dot_z_v = torch.matmul(u, v.T)
/tmp/ipykernel_11037/371701000.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sigma_1 = torch.tensor(neg_indices[i], requires_grad=True)
/tmp/ipykernel_11037/371701000.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sigma_2 = torch.tensor(pos_indi

Epoch:1, Loss:727.708740234375, Train:0.5684931506849316, Test:0.5675675675675675
latent space shape: torch.Size([183, 5])
Epoch:2, Loss:227922.3125, Train:0.684931506849315, Test:0.5945945945945946
latent space shape: torch.Size([183, 5])
Epoch:3, Loss:937.677490234375, Train:0.7191780821917808, Test:0.5675675675675675
latent space shape: torch.Size([183, 5])
Epoch:4, Loss:853.1014404296875, Train:0.7808219178082192, Test:0.5135135135135135
latent space shape: torch.Size([183, 5])
Epoch:5, Loss:717.2266845703125, Train:0.7602739726027398, Test:0.5135135135135135
latent space shape: torch.Size([183, 5])
Epoch:6, Loss:747.3983764648438, Train:0.7876712328767124, Test:0.5135135135135135
latent space shape: torch.Size([183, 5])
Epoch:7, Loss:671.7625122070312, Train:0.821917808219178, Test:0.4864864864864865
latent space shape: torch.Size([183, 5])
Epoch:8, Loss:654.3624877929688, Train:0.8561643835616438, Test:0.4864864864864865
latent space shape: torch.Size([183, 5])
Epoch:9, Loss:693.

kl_loss,▂▂▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▄▄▄▄▄▄▄▄▄▄▄▇███▇▇
mean_test_acc,▁
mean_train_loss,▁
nc_loss,▄▄▃▄▃▃▄▃█▄▃▂▂▂▂▃▄▃▂▂▂▁▁▂▃▄▂▅▃▃▃▂▃▃▃▁▁▂▁▃
neg_indices,▅▁█▂▄▄▅▄▂▆▂▅▄▃▃▄▂▂▁▄▂▄▃▃▆▃▃▄▃▂▃▅▃▂▇▁▂▃▂▆
obj_loss,▃▂▄▄▂▂▃▃▄▃▂▂▃▃▂▃▃▂▃▄▂▃▃▂▃▄▂▂▂▂▂▂▂▃▃█▂▁▁▅
pos_indices,▂▂▂▂▂▃▂▅▂▂▂▄▂▃▂▁▂▂▂▁▂▂▂▁▅▃▂▃▂▂▅▂▂▁█▄▂▃▂▂
recon_loss,▂▂▁▂▂▄▄▄▁▄▄▅▃▁▂▄▂▃▃▂▃▃▄▂▁▃▁▃▁▂▂▂▄▃▂█▁▂▂▃
test_acc,▃▂▂▃▃▃▃▂▇▄▅▂▂▁▂▁█▆▅▃▄▂▃▃▅▃▃▄▃▃▃▄▆▅▄▃▃▂▃▂
test_std,▁
total_loss,▃▂▄▄▂▂▃▃▄▃▂▂▃▃▂▃▃▂▃▄▂▃▃▂▃▄▂▂▂▂▂▂▂▃▃█▂▁▁▅


In [ ]:
run_vgae_rewire.finish()

kl_loss,▁█▁▁▂▂▂▂▃▃▃▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▆▆▆▇▆▆▆▆▇▆▇▇▇
nc_loss,█▆▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
neg_indices,▂█▄▁▁▁▂▃▁▄▁▂▂▂▁▄▃▂▂▄▂▁▂▁▅▂▂▂▂▃▅▂▂▁▂▂▂▃▂▂
obj_loss,▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▂▁▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂
pos_indices,▂▂▄▂▂▂▂▃▂▂▂▃▂▄▂▂▂▂▂▂▄▁▆▂▂▅▃▂▂█▂▁▄▂▂▂▃▂▄▄
recon_loss,▂█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_acc,▁▅▇█████████████████████████████████████
total_loss,▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▂▁▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂
train_acc,▁▅▇█████████████████████████████████████
val_acc,▁▅▆▇▆▆▆▆▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇██▇▇▇▇▇▇▇▇▇▇█▇▇▇▇
kl_loss,0.00326
